In [25]:
import pandas as pd
import star_players


def classify_player(row):
    if row['display_first_last'] in (star_players.get_star_players()):
        return 'STAR'
    elif row['to_year'] - row['from_year'] >= 6:
        return 'SOLID'
    else:
        return 'BUST'

dataframe = pd.read_csv("common_player_info.csv")
draft_year = 1995

dataframe = dataframe[dataframe["from_year"] >= draft_year]
dataframe.sort_values(by=['from_year'], inplace=True)


dataframe['classification'] = dataframe.apply(classify_player, axis=1)

dataframe = dataframe[
    (dataframe['from_year'] >= 2017) &
    (dataframe['to_year'] >= 2023) &
    (dataframe['classification'] == 'BUST')
    ]

#REMOVE ALL PLAYERS IN THE ABOVE DATAFRAME FROM THE STUDY

# dataframe = dataframe[(dataframe['classification'] != 'BUST') & (dataframe['to_year'] - dataframe['from_year'] < 5)]

columns = ['display_first_last', 'classification', 'from_year', 'to_year']

dataframe = dataframe[columns]


dataframe.to_csv('output.csv', index=False)














